# Stanford CME 241 (Winter 2026) - Assignment 2

**Due: Friday, February 13 @ 11:59 PM PST on Gradescope.**

Assignment instructions:
- Make sure each of the subquestions have answers
- Ensure that group members indicate which problems they're in charge of
- Show work and walk through your thought process where applicable
- Empty code blocks are for your use, so feel free to create more under each section as needed
- Document code with light comments (i.e. 'this function handles visualization')

Submission instructions:
- When complete, fill out your publicly available GitHub repo file URL and group members below, then export or print this .ipynb file to PDF and upload the PDF to Gradescope.

*Link to this ipynb file in your public GitHub repo (replace below URL with yours):* 

https://github.com/my-username/my-repo/assignment-file-name.ipynb

*Group members (replace below names with people in your group):* 
- Person 1
- Person 2
- Person 3

## Imports

In [20]:
import numpy as np
import matplotlib.pyplot as plt

## Question 1: Job-Hopping and Wages-Utility-Maximization (Led by ______)

You are a worker who starts every day either employed or unemployed. If you start your day employed, you work on your job for the day (one of $n$ jobs, as elaborated later) and you get to earn the wage of the job for the day. However, at the end of the day, you could lose your job with probability $\alpha \in [0,1]$, in which case you start the next day unemployed. If at the end of the day, you do not lose your job (with probability $1-\alpha$), then you will start the next day with the same job (and hence, the same daily wage). 

On the other hand, if you start your day unemployed, then you will be randomly offered one of $n$ jobs with daily wages $w_1, w_2, \ldots w_n \in \mathbb{R}^+$ with respective job-offer probabilities $p_1, p_2, \ldots p_n \in [0,1]$ (with $\sum_{i=1}^n p_i = 1$). You can choose to either accept or decline the offered job. If you accept the job offer, your day progresses exactly like the **employed-day** described above (earning the day's job wage and possibly (with probability $\alpha$) losing the job at the end of the day). However, if you decline the job offer, you spend the day unemployed, receive the unemployment wage $w_0 \in \mathbb{R}^+$ for the day, and start the next day unemployed.

The problem is to identify the optimal choice of accepting or rejecting any of the job offers the worker receives, in a manner that maximizes the infinite-horizon **Expected Discounted-Sum of Wages Utility**. Assume the daily discount factor for wages (employed or unemployed) is $\gamma \in [0,1])$. Assume Wages Utility function to be $U(w) = \log(w)$ for any wage amount $w \in \mathbb{R}^+$. The goal is to maximize:

$$
\mathbb{E}\left[\sum_{u=t}^\infty \gamma^{u-t} \cdot \log(w_{i_u})\right]
$$

at the start of a given day $t$ ($w_{i_u}$ is the wage earned on day $u$, $0 \leq i_u \leq n$ for all $u \geq t$).

---

### Subquestions

#### Part (A): MDP Modeling

Express the job-hopping problem as an MDP using clear mathematical notation by defining the following components:

1. **State Space**: Define the possible states of the MDP.
2. **Action Space**: Specify the actions available to the worker at each state.
3. **Transition Function**: Describe the probabilities of transitioning between states for each action.
4. **Reward Function**: Specify the reward associated with the states and transitions.
5. **Bellman Optimality Equation**: Write the Bellman Optimality Equation customized for this MDP.

---

#### Part (B): Python Implementation

Write Python code that:

1. Solves the Bellman Optimality Equation (hence, solves for the **Optimal Value Function** and the **Optimal Policy**) with a numerical iterative algorithm. 
2. Clearly define the inputs and outputs of your algorithm with their types (`int`, `float`, `List`, `Mapping`, etc.).

*Note*: For this problem, write the algorithm from scratch without using any prebuilt MDP/DP libraries or code.

---

#### Part (C): Visualization and Analysis

1. Plot the **Optimal Value Function** as a function of the state for a specific set of parameters ($n$, $w_1, \ldots, w_n$, $p_1, \ldots, p_n$, $\alpha$, $\gamma$, $w_0$).
2. Include these graphs in your submission.

---

#### Part (D): Observations

1. What patterns do you observe in the **Optimal Policy** as you vary the parameters $n$, $\alpha$, and $\gamma$?
2. Provide a brief discussion of your findings.

---

### Part (A) Answer

<span style="color:red">*fill in*</span>

### Part (B) Answer

In [7]:
# fill in with Python code

### Part (C) Answer

<span style="color:red">*fill in*</span>

### Part (D) Answer

<span style="color:red">*fill in*</span>

## Question 2: Two-Stores Inventory Control (Led by ______)

We extend the capacity-constrained inventory example implemented in [rl/chapter3/simple_inventory_mdp_cap.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/chapter3/simple_inventory_mdp_cap.py) as a `FiniteMarkovDecisionProcess` (the Finite MDP model for the capacity-constrained inventory example is described in detail in Chapters 1 and 2 of the RLForFinanceBook). Here we assume that we have two different stores, each with their own separate capacities $C_1$ and $C_2$, their own separate Poisson probability distributions of demand (with means $\lambda_1$ and $\lambda_2$), their own separate holding costs $h_1$ and $h_2$, and their own separate stockout costs $p_1$ and $p_2$. At 6pm upon stores closing each evening, each store can choose to order inventory from a common supplier (as usual, ordered inventory will arrive at the store 36 hours later). We are also allowed to transfer inventory from one store to another, and any such transfer happens overnight, i.e., will arrive by 6am next morning (since the stores are fairly close to each other). Note that the orders are constrained such that following the orders on each evening, each store's inventory position (sum of on-hand inventory and on-order inventory) cannot exceed the store's capacity (this means the action space is constrained to be finite). Each order made to the supplier incurs a fixed transportation cost of $K_1$ (fixed-cost means the cost is the same no matter how many units of non-zero inventory a particular store orders). Moving any non-zero inventory between the two stores incurs a fixed transportation cost of $K_2$. 

Model this as a derived class of `FiniteMarkovDecisionProcess` much like we did for `SimpleInventoryMDPCap` in the code repo. Set up instances of this derived class for different choices of the problem parameters (capacities, costs etc.), and determine the Optimal Value Function and Optimal Policy by invoking the function `value_iteration` (or `policy_iteration`) from file [rl/dynamic_programming.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/dynamic_programming.py).

Analyze the obtained Optimal Policy and verify that it makes intuitive sense as a function of the problem parameters.

In [ ]:
# fill in with Python code

## Question 3: Dynamic Price Optimization (Led by Jeffrey Xue)

You own a supermarket, and you are $T$ days away from Halloween 🎃. You have just received $M$ Halloween masks from your supplier. You want to dynamically set the selling price of the Halloween masks at the start of each day in a manner that maximizes your **Expected Total Sales Revenue** for Halloween masks this season (assume no one will buy Halloween masks after Halloween).

Assume that for each of the $T$ days, you are required to select a price for that day from one of $N$ prices $p_1, p_2, \dots, p_N \in \mathbb{R}$, and that price is the selling price for all masks on that day. Assume that the customer demand for the number of Halloween masks on any day is governed by a Poisson probability distribution with mean $\lambda_i \in \mathbb{R}$ if you select that day’s price to be $p_i$ (where $i$ is a choice among $1, 2, \dots, N$).

Note that on any given day, the demand could exceed the number of Halloween masks you have in the store, in which case the number of masks sold on that day will be equal to the number of Halloween masks you had at the start of that day.

We spoke about this example in class - referencing the slides here (if needed) could be helpful!

---

### Subquestions

#### Part (A): Bellman Optimality Equation

Write the **Bellman Optimality Equation** customized to this Markov Decision Process (MDP). Essentially, you need to express the **Optimal Value Function** $v_*$ recursively based on taking the best action in the current state and based on the subsequent random customer demand that would produce the appropriate reward and take you to the next state.

**Note**: The probability mass function of a Poisson distribution with mean $\lambda \in \mathbb{R}$ is given by:

$$
f(k) = \frac{e^{-\lambda} \lambda^k}{k!}, \quad k = 0, 1, 2, \dots
$$

---

#### Part (B): Boundary Conditions

To be able to solve the $v_*$ recursion, you need to know the values of $v_*$ for the boundary case (boundary states). Write down the boundary case(s) for the $v_*$ recursion.

---

#### Part (C): Numerical Solution

You can solve this $v_*$ recursion (hence, solve for the **Optimal Policy** $\pi_*$) with a numerical recursive algorithm (essentially a special form of Dynamic Programming algorithm customized to this problem). 

Write Python code for this algorithm that would enable you to dynamically set the selling price at the start of each day. Clearly define the inputs and outputs of your algorithm with their types (`int`, `float`, `List`, `Mapping`, etc.).

---


### Part (A) Answer

Let's define the state as $(t, I_t)$ where $0 \leq t \leq T$ is the day number starting at 0 until day $T$ which is Halloween and $0 \leq I_t \leq M$ is the current inventory of masks on the current day. We can define the action space as the choice of a price $p_i$ for $1 \leq i \leq N$. If $t = T$ or $m = 0$, no action is possible so the action space is the empty set. The sales revenue is $\min(I_t, d_t) \cdot p_i$ where $d_t \sim \text{Poisson}(\lambda_i)$, in other words if demand is less than inventory (i.e. $d_t < I_t$) the revenue is $d_tp_i$ and if demand is at least the inventory (i.e. $d_t \geq I_t$) the revenue is $I_tp_i$.

With this setup, the Bellman Optimality Equation is:

$$
v_*(t, I_t) = \max_{i \in \{1,\ldots,N\}} \left\{ (\text{expected revenue when } d_t < I_t )+ (\text{expected revenue when } d_t \geq I_t) \right\} \\
v_*(t, I_t) = \max_{i \in \{1,\ldots,N\}} \left\{ \sum_{d=0}^{I_t-1} \Pr(Poi(\lambda_i) = d) \Big[ p_i \cdot d + v_*(t+1,\, I_t-d) \Big] \;+\; \left(1 - \sum_{d=0}^{I_t-1} \Pr(Poi(\lambda_i) = d)\right) \Big[ p_i \cdot I_t + v_*(t+1,\, 0) \Big] \right\} \\
v_*(t, I_t) = \max_{i \in \{1,\ldots,N\}} \left\{ \sum_{d=0}^{I_t-1} \frac{e^{-\lambda_i}\lambda_i^d}{d!} \Big[ p_i \cdot d + v_*(t+1,\, I_t-d) \Big] \;+\; \left(1 - \sum_{d=0}^{I_t-1} \frac{e^{-\lambda_i}\lambda_i^d}{d!}\right) \Big[ p_i \cdot I_t + v_*(t+1,\, 0) \Big] \right\} \\
$$

### Part (B) Answer

When there are no days left ($t = T$), no more sales can be made regardless of inventory, so:

$$v_*(0, I_t) = 0 \quad \text{for all } I_t \in \{0, 1, \ldots, M\}.$$

When there are no masks left to sell ($I_t = 0$), no revenue can be earned regardless of how many days remain, so:

$$v_*(t, 0) = 0 \quad \text{for all } t \in \{0, 1, \ldots, T\}.$$

### Part (C) Answer

In [1]:
from typing import List, Tuple, Dict
from scipy.stats import poisson
import numpy as np

# inputs are the T, M, a list of prices, and a list of lambdas.
# outputs are the optimal expected total revenue (ie optimal value function V) and the optimal policy for each state.
def dp_recursive_algorithm(T: int, M: int, prices: List[float], lambdas: List[float]) -> Tuple[np.ndarray, Dict[Tuple[int, int], int]]:
    N = len(prices)
    V = np.zeros((T + 1, M + 1))
    policy: Dict[Tuple[int, int], int] = {}
    for t in range(1, T + 1):
        for m in range(1, M + 1):
            best_val = -np.inf
            best_action = 0
            for i in range(N):
                lam = lambdas[i]
                p = prices[i]
                expected_val = 0.0
                for d in range(m):
                    prob_d = poisson.pmf(d, lam)
                    expected_val += prob_d * (p * d + V[t - 1][m - d])
                prob_ge_m = 1.0 - poisson.cdf(m - 1, lam)
                expected_val += prob_ge_m * (p * m + V[t - 1][0])
                if expected_val > best_val:
                    best_val = expected_val
                    best_action = i
            V[t][m] = best_val
            policy[(t, m)] = best_action
    return V, policy

# example 
T = 10
M = 30
prices =  [5.0, 8.0, 10.0, 12.0, 15.0]
lambdas = [12.0, 8.0, 6.0, 4.0, 2.0]
V, policy = dp_recursive_algorithm(T, M, prices, lambdas)
print(f"Optimal expected total revenue (T={T}, M={M}): ${V[T][M]:.2f}\n")

Optimal expected total revenue (T=10, M=30): $378.45



## Question 4: Risk-Aversion and Utility Optimization under CARA Utility (Led by Patrick Flanagan)

You are tasked with analyzing the behavior of an investor who seeks to maximize their utility under **CARA Utility**. The investor has wealth $W$ and the CARA utility function:

$$
U(W) = \frac{1 - e^{-aW}}{a}, \quad a > 0,
$$

where $a$ represents the investor's **risk aversion**.

The investor allocates their wealth between:
1. A **riskless asset** with a fixed return $r$, and
2. A **risky asset** with return $R \sim \mathcal{N}(\mu, \sigma^2)$

The investor allocates a fraction $\pi$ of their wealth to the risky asset and $(1 - \pi)$ to the riskless asset. The wealth $W$ after one year is given by:

$$
W = (1 + r)(1 - \pi) + (1 + R)\pi.
$$

The goal is to analyze the investor’s optimal allocation $\pi$ to the risky asset and compute key risk-related quantities.

---

### Subquestions

#### Part (A): Expected Utility and Certainty-Equivalent Wealth

1. Derive the expression for the **expected utility** $\mathbb{E}[U(W)]$, using the given CARA utility function and assuming $R \sim \mathcal{N}(\mu, \sigma^2)$.
2. Using a Taylor expansion, approximate the **certainty-equivalent wealth** $W_{CE}$ up to second-order terms.

---

#### Part (B): Optimal Portfolio Allocation

Derive the optimal fraction $\pi^*$ of wealth to be allocated to the risky asset such that the **expected utility** $\mathbb{E}[U(W)]$ is maximized. Express $\pi^*$ in terms of $a$, $\mu$, $r$, and $\sigma^2$.

---

#### Part (C): Risk Premium

1. Using the results from Part (A), calculate the **absolute risk premium** $\pi_A = \mathbb{E}[W] - W_{CE}$.
2. Verify that $\pi_A \approx \frac{a \pi^2 \sigma^2}{2}$ for small $\sigma^2$.

---

#### Part (D): Numerical Calculations and Interpretation

Given the parameters $r = 0.02$, $\mu = 0.08$, $\sigma^2 = 0.04$, and $a = 3$:
1. Compute the optimal allocation $\pi^*$.
2. Calculate the certainty-equivalent wealth $W_{CE}$.
3. Compute the absolute risk premium $\pi_A$.
4. Interpret the results and discuss how changes in $a$ and $\sigma^2$ affect the risk premium and portfolio allocation.

---

#### Part (E): Expected Utility under Uniform Distribution

Now assume that the return of the risky asset, $R$, is no longer normally distributed. Instead, $R \sim \text{Uniform}[\alpha, \beta]$, where $\alpha$ and $\beta$ are the lower and upper bounds of the distribution, respectively.

1. Derive the new expression for the **expected utility** $\mathbb{E}[U(W)]$. Make sure to simplify your result as much as possible, and ensure that it explicitly depends on $a$, $\pi$, $\alpha$, $\beta$, and $r$.

**Hint**: Use the fact that if $W \sim \text{Uniform}[w_{\text{min}}, w_{\text{max}}]$, then:

$$
\mathbb{E}[g(W)] = \frac{1}{w_{\text{max}} - w_{\text{min}}} \int_{w_{\text{min}}}^{w_{\text{max}}} g(W) \, dW.
$$

---

### Part (A) Answer

**Simplify terminal wealth:**

$$W = (1+r)(1-\pi) + (1+R)\pi = (1+r) + \pi(R - r)$$

Since $R \sim \mathcal{N}(\mu, \sigma^2)$, terminal wealth $W$ is normally distributed:

$$W \sim \mathcal{N}\big((1+r) + \pi(\mu - r),\; \pi^2\sigma^2\big)$$

**Derive $\mathbb{E}[U(W)]$:**

$$\mathbb{E}[U(W)] = \frac{1 - \mathbb{E}[e^{-aW}]}{a}$$

Using the moment generating function of a normal random variable, $\mathbb{E}[e^{tX}] = e^{t\mathbb{E}[X] + \frac{t^2}{2}\text{Var}(X)}$, with $t = -a$:

$$\mathbb{E}[e^{-aW}] = \exp\left(-a\left[(1+r) + \pi(\mu - r)\right] + \frac{a^2 \pi^2 \sigma^2}{2}\right)$$

Therefore:

$$\mathbb{E}[U(W)] = \frac{1 - \exp\left(-a\left[(1+r) + \pi(\mu - r)\right] + \frac{a^2 \pi^2 \sigma^2}{2}\right)}{a}$$

**Certainty-equivalent wealth via Taylor expansion:**

Expand $U(W)$ around $\mathbb{E}[W]$ to second order:

$$U(W) \approx U(\mathbb{E}[W]) + U'(\mathbb{E}[W])(W - \mathbb{E}[W]) + \frac{1}{2}U''(\mathbb{E}[W])(W - \mathbb{E}[W])^2$$

Taking expectations:

$$\mathbb{E}[U(W)] \approx U(\mathbb{E}[W]) + \frac{1}{2}U''(\mathbb{E}[W])\,\text{Var}(W)$$

Since $W_{CE}$ satisfies $U(W_{CE}) = \mathbb{E}[U(W)]$, expanding $U(W_{CE})$ around $\mathbb{E}[W]$ to first order and equating:

$$W_{CE} \approx \mathbb{E}[W] + \frac{1}{2}\frac{U''(\mathbb{E}[W])}{U'(\mathbb{E}[W])}\,\text{Var}(W)$$

For CARA utility, $U'(W) = e^{-aW}$ and $U''(W) = -ae^{-aW}$, so $\frac{U''}{U'} = -a$. Therefore:

$$\boxed{W_{CE} \approx \mathbb{E}[W] - \frac{a}{2}\,\text{Var}(W) = (1+r) + \pi(\mu - r) - \frac{a\pi^2\sigma^2}{2}}$$

Note: for CARA utility with normally distributed wealth, this second-order approximation is exact.


### Part (B) Answer

Maximizing $\mathbb{E}[U(W)]$ is equivalent to maximizing the quantity inside the exponential (with sign flipped), which from Part (A) gives us:

$$\max_\pi \left[(1+r) + \pi(\mu - r) - \frac{a\pi^2\sigma^2}{2}\right]$$

This is the same as maximizing $W_{CE}$ with respect to $\pi$. Taking the first-order condition:

$$\frac{\partial}{\partial \pi}: \quad (\mu - r) - a\pi\sigma^2 = 0$$

Solving for $\pi$:

$$\boxed{\pi^* = \frac{\mu - r}{a\sigma^2}}$$

The second-order condition confirms this is a maximum since $\frac{\partial^2}{\partial \pi^2} = -a\sigma^2 < 0$.

**Intuition:** The optimal risky allocation increases with the excess return $(\mu - r)$ and decreases with both risk aversion $(a)$ and return variance $(\sigma^2)$.


### Part (C) Answer

**Absolute risk premium:**

$$\pi_A = \mathbb{E}[W] - W_{CE} = \left[(1+r) + \pi(\mu-r)\right] - \left[(1+r) + \pi(\mu-r) - \frac{a\pi^2\sigma^2}{2}\right]$$

$$\boxed{\pi_A = \frac{a\pi^2\sigma^2}{2}}$$

**Verification:** The Arrow-Pratt approximation states that for small risks, $\pi_A \approx \frac{1}{2}A(W)\,\text{Var}(W)$, where $A(W) = -\frac{U''(W)}{U'(W)}$ is the absolute risk aversion coefficient. For CARA utility, $A(W) = a$ (constant), and $\text{Var}(W) = \pi^2\sigma^2$, giving:

$$\pi_A \approx \frac{1}{2}\cdot a \cdot \pi^2\sigma^2 = \frac{a\pi^2\sigma^2}{2}$$

This matches exactly because CARA utility with normally distributed wealth makes the second-order Taylor approximation exact — all higher-order terms are absorbed by the exponential-normal pairing.


### Part (D) Answer

Given $r = 0.02$, $\mu = 0.08$, $\sigma^2 = 0.04$, $a = 3$:

**1. Optimal allocation:**

$$\pi^* = \frac{\mu - r}{a\sigma^2} = \frac{0.08 - 0.02}{3 \times 0.04} = \frac{0.06}{0.12} = 0.5$$

The investor allocates 50% of wealth to the risky asset.

**2. Certainty-equivalent wealth:**

$$W_{CE} = (1+r) + \pi^*(\mu - r) - \frac{a(\pi^*)^2\sigma^2}{2} = 1.02 + 0.5(0.06) - \frac{3(0.25)(0.04)}{2} = 1.035$$

**3. Absolute risk premium:**

$$\pi_A = \frac{a(\pi^*)^2\sigma^2}{2} = \frac{3(0.25)(0.04)}{2} = 0.015$$

**4. Interpretation:**

- The investor places half their wealth in the risky asset, reflecting a moderate balance between the excess return $(\mu - r = 6\%)$ and the penalty from risk aversion and variance.
- The risk premium of $0.015$ means the investor would accept a guaranteed return 1.5% lower than the expected return to eliminate all risk.
- **Increasing $a$ (more risk-averse):** $\pi^*$ decreases (invest less in risky asset) and $\pi_A$ increases (demand a larger premium for bearing risk), since the investor is more sensitive to variance.
- **Increasing $\sigma^2$ (more volatile):** $\pi^*$ also decreases (risky asset is less attractive) and $\pi_A$ increases (more uncertainty means a larger gap between expected wealth and the certainty equivalent).


### Part (E) Answer

With $R \sim \text{Uniform}[\alpha, \beta]$, terminal wealth $W = (1+r) + \pi(R - r)$ is uniformly distributed on $[W_{\min},\, W_{\max}]$ where:

$$W_{\min} = (1+r) + \pi(\alpha - r), \quad W_{\max} = (1+r) + \pi(\beta - r)$$

Using the hint, compute $\mathbb{E}[e^{-aW}]$:

$$\mathbb{E}[e^{-aW}] = \frac{1}{W_{\max} - W_{\min}} \int_{W_{\min}}^{W_{\max}} e^{-aW}\,dW = \frac{1}{W_{\max} - W_{\min}} \left[\frac{-1}{a}e^{-aW}\right]_{W_{\min}}^{W_{\max}}$$

$$= \frac{e^{-aW_{\min}} - e^{-aW_{\max}}}{a(W_{\max} - W_{\min})}$$

Since $W_{\max} - W_{\min} = \pi(\beta - \alpha)$:

$$\mathbb{E}[e^{-aW}] = \frac{e^{-a[(1+r)+\pi(\alpha - r)]} - e^{-a[(1+r)+\pi(\beta - r)]}}{a\pi(\beta - \alpha)}$$

Substituting into $\mathbb{E}[U(W)] = \frac{1 - \mathbb{E}[e^{-aW}]}{a}$:

$$\boxed{\mathbb{E}[U(W)] = \frac{1}{a}\left(1 - \frac{e^{-a[(1+r)+\pi(\alpha - r)]} - e^{-a[(1+r)+\pi(\beta - r)]}}{a\pi(\beta - \alpha)}\right)}$$

This expression depends explicitly on $a$, $\pi$, $\alpha$, $\beta$, and $r$ as required. Note that the Gaussian MGF trick no longer applies — instead we directly integrated the exponential over the uniform support to obtain the closed-form result.
